In [20]:
import pandas as pd
import numpy as np
import statistics
import csv
from sklearn import metrics
from xgboost import XGBClassifier
import random
import math
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier

In [18]:
#Extracting The Spatial Feature

def spatial_two (Node_class, Adj,n):
    F_vec=[]
    for i in range(n):
        #print("\rProcessing file {} ({}%)".format(i, 100*i//(n-1)), end='', flush=True)
        node_F=[]
        list_out=[]
        list_In=[]
        S_nbd_out=[]
        S_nbd_in=[]
        for j in range(n):
            if Adj[i][j]==1 and i!=j:
                list_out.append(Adj[j][j])
                for k in range(n):
                    if A[j][k]==1 and k!=j:
                        S_nbd_out.append(Adj[k][k])
            if Adj[j][i]==1 and i!=j:
                list_In.append(A[j][j])
                for k in range(n):
                    if Adj[k][j]==1 and k!=j:
                        S_nbd_in.append(Adj[k][k])

        for d in Node_class:
            count=0
            count_in=0
            for k in range(len(list_out)):
                if list_out[k]==d:
                    count=count+1
            node_F.append(count)
            for k in range(len(list_In)):
                if list_In[k]==d:
                    count_in=count_in+1
            node_F.append(count_in)
        for d in Node_class:
            count_S_out=0
            count_S_in=0
            for k in range(len(S_nbd_out)):
                if S_nbd_out[k]==d:
                    count_S_out=count_S_out+1
            node_F.append(count_S_out)
            for k in range(len(S_nbd_in)):
                if S_nbd_in[k]==d:
                    count_S_in=count_S_in+1
            node_F.append(count_S_in)
        F_vec.append(node_F)
    return F_vec

# Wisconsin

In [217]:
from urllib.request import urlopen
p_data=urlopen("https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/WebKb/Wisconsin/out1_graph_edges.txt")
er=[]
for line in p_data:
    output = line.decode()
    er.append(output)
myls=list(er)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
print(mylist)

['63\t78', '63\t92', '27\t63', '27\t111', '110\t12', '98\t63', '98\t27', '98\t244', '98\t147', '98\t86', '98\t163', '98\t132', '98\t175', '98\t19', '98\t17', '98\t190', '98\t102', '98\t114', '98\t170', '98\t243', '98\t6', '98\t57', '98\t67', '98\t211', '98\t29', '98\t33', '98\t144', '98\t109', '98\t96', '98\t204', '98\t52', '98\t59', '98\t60', '98\t4', '98\t131', '98\t174', '98\t113', '98\t82', '98\t25', '98\t54', '98\t138', '98\t213', '98\t41', '98\t177', '98\t171', '98\t172', '98\t125', '98\t8', '98\t123', '98\t145', '98\t111', '98\t139', '98\t137', '98\t76', '98\t45', '98\t238', '98\t15', '98\t153', '98\t32', '98\t235', '98\t230', '98\t237', '98\t192', '98\t168', '98\t173', '98\t28', '98\t104', '98\t124', '98\t135', '98\t2', '98\t68', '98\t205', '98\t169', '98\t229', '98\t88', '98\t16', '98\t157', '98\t1', '98\t227', '98\t154', '98\t10', '98\t184', '98\t22', '98\t75', '98\t220', '98\t216', '98\t12', '98\t80', '98\t146', '98\t95', '98\t209', '98\t94', '98\t133', '98\t187', '98\t55', 

In [218]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
print(node_ID)
n=len(node_ID)
print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [219]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [220]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/WebKb/Wisconsin/out1_node_feature_label.txt',sep='\s+')
df.head()


,node_id,feature,label
0,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...",1
1,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2
2,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2
3,3,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2
4,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",1


In [221]:
fe=df["feature"].to_numpy()
lst_Class=df["label"].to_numpy()
lst_ID=df["node_id"].to_numpy()

lst_Fe=[]
for row in fe:
    d=row.split(',')
    fev=[]
    for j in range(len(d)):
            fev.append(int(d[j]))
    lst_Fe.append(fev)


In [222]:
feature_names = ["w_{}".format(ii) for ii in range(1703)]
Data=pd.DataFrame(lst_Fe,columns=feature_names)
Data.insert(loc=1703,column='Class',value=lst_Class)
Data.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [223]:
Class=lst_Class.copy()

In [224]:
#CLUSTERING DATA ACCORDING TO CLASS
Data_0=Data.loc[Data['Class'] == 0]
Data_1=Data.loc[Data['Class'] == 1]
Data_2=Data.loc[Data['Class'] == 2]
Data_3=Data.loc[Data['Class'] == 3]
Data_4=Data.loc[Data['Class'] == 4]
Data_1.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [225]:
# FINDING THE INCLUSIVE REPRESENTATIVE

Basis_0=[]
Basis_1=[]
Basis_2=[]
Basis_3=[]
Basis_4=[]
for i in range(1703):
    W="w_{}".format(i)
    Basis_0.append(max(Data_0[W].to_numpy()))
    Basis_1.append(max(Data_1[W].to_numpy()))
    Basis_2.append(max(Data_2[W].to_numpy()))
    Basis_3.append(max(Data_3[W].to_numpy()))
    Basis_4.append(max(Data_4[W].to_numpy()))
#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(Basis_0)
count0=0
count1=0
for i in range(1703):
    if Basis_0[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 

In [226]:
# FINDING THE SELECTIVE REPRESENTATIVE

SBasis_0=[]
SBasis_1=[]
SBasis_2=[]
SBasis_3=[]
SBasis_4=[]
#for i in range(14):
 #   W="w_{}".format(i)
  #  print(list(Data_0[W].to_numpy()).count(1))
for i in range(1703):
    W="w_{}".format(i)
    if list(Data_0[W].to_numpy()).count(1)>=int(len(Data_0.index)*0.1):
        SBasis_0.append(1)
    else:
        SBasis_0.append(0) 
    if list(Data_1[W].to_numpy()).count(1)>=int(len(Data_1.index)*0.1):
        SBasis_1.append(1)
    else:
        SBasis_1.append(0)
    if list(Data_2[W].to_numpy()).count(1)>=int(len(Data_2.index)*0.1):
        SBasis_2.append(1)
    else:
        SBasis_2.append(0)
    if list(Data_3[W].to_numpy()).count(1)>=int(len(Data_3.index)*0.1):
        SBasis_3.append(1)
    else:
        SBasis_3.append(0)
    if list(Data_4[W].to_numpy()).count(1)>=int(len(Data_4.index)*0.1):
        SBasis_4.append(1)
    else:
        SBasis_4.append(0)

#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(SBasis_3)
count0=0
count1=0
for i in range(1703):
    if SBasis_2[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 

In [227]:
# FEATURE VECTOR USING INCLUSIVE REPRESENTATIVE

Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(1703):
        if f[j]==1 and Basis_0[j]==1:
            count=count+1;
        if f[j]==1 and Basis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and Basis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and Basis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and Basis_4[j]==1:
            count4=count4+1;

    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)

    #print(f)
    f.clear()
    Fec.append(vec)
print(Fec)

[[46, 72, 68, 59, 56], [39, 47, 49, 45, 45], [42, 48, 51, 49, 48], [27, 32, 33, 29, 27], [36, 64, 63, 58, 43], [81, 145, 129, 122, 103], [131, 242, 208, 176, 165], [35, 48, 48, 45, 37], [52, 69, 78, 67, 61], [85, 82, 83, 78, 72], [41, 43, 47, 45, 42], [52, 65, 70, 67, 60], [45, 53, 60, 55, 50], [50, 62, 74, 62, 58], [37, 42, 42, 42, 38], [50, 53, 63, 60, 56], [45, 47, 54, 45, 43], [36, 39, 41, 37, 34], [47, 61, 66, 66, 52], [22, 26, 26, 24, 22], [30, 48, 46, 39, 38], [79, 129, 115, 92, 88], [26, 30, 33, 28, 25], [44, 56, 67, 60, 56], [61, 97, 88, 76, 64], [82, 113, 116, 126, 99], [55, 94, 86, 83, 66], [81, 120, 126, 141, 120], [76, 142, 122, 112, 100], [46, 56, 69, 63, 59], [14, 14, 14, 14, 14], [75, 92, 108, 105, 92], [65, 72, 74, 65, 62], [28, 34, 36, 33, 28], [113, 205, 180, 177, 159], [138, 249, 220, 179, 168], [36, 74, 70, 68, 60], [83, 100, 114, 118, 98], [90, 152, 126, 112, 108], [36, 39, 41, 41, 38], [42, 49, 59, 44, 46], [55, 60, 67, 61, 57], [47, 84, 72, 69, 60], [42, 54, 70,

In [228]:
# FEATURE VECTOR USING SELECTIVE REPRESENTATIVE

SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0

    for j in range(1703):
        if f[j]==1 and SBasis_0[j]==1:
            count=count+1;
        if f[j]==1 and SBasis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and SBasis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and SBasis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and SBasis_4[j]==1:
            count4=count4+1;

    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)

    #print(f)
    f.clear()
    SFec.append(Svec)
print(SFec)

[[46, 63, 40, 52, 44], [39, 38, 40, 44, 38], [42, 34, 40, 43, 42], [27, 27, 27, 26, 24], [36, 54, 36, 43, 35], [81, 105, 54, 81, 78], [131, 174, 63, 104, 115], [35, 35, 27, 36, 32], [52, 49, 48, 54, 49], [85, 55, 43, 60, 59], [41, 35, 39, 41, 39], [52, 42, 47, 59, 52], [45, 32, 39, 46, 41], [50, 43, 41, 55, 45], [37, 33, 37, 39, 35], [50, 40, 47, 56, 46], [45, 35, 37, 32, 37], [36, 30, 36, 32, 29], [47, 42, 41, 52, 43], [22, 21, 22, 21, 22], [30, 38, 25, 32, 30], [79, 103, 44, 66, 62], [26, 25, 28, 23, 22], [44, 37, 37, 56, 49], [61, 86, 38, 52, 45], [82, 68, 50, 97, 75], [55, 78, 39, 56, 53], [81, 53, 47, 118, 95], [76, 113, 45, 76, 70], [46, 40, 47, 61, 48], [14, 14, 14, 14, 14], [75, 56, 58, 93, 78], [65, 49, 49, 52, 51], [28, 29, 31, 28, 27], [113, 121, 58, 126, 123], [138, 182, 64, 105, 118], [36, 55, 31, 47, 44], [83, 56, 56, 105, 85], [90, 116, 43, 76, 82], [36, 34, 34, 37, 32], [42, 34, 36, 31, 35], [55, 43, 53, 55, 50], [47, 69, 40, 52, 47], [42, 32, 32, 42, 41], [75, 60, 45, 

In [229]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(4):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4
0,46,63,40,52,44
1,39,38,40,44,38
2,42,34,40,43,42
3,27,27,27,26,24
4,36,54,36,43,35


In [230]:
y =np.array(Fec)
data1=pd.DataFrame({"In_0":y[:,0]})
for i in range(4):
    data1.insert(loc=i+1, column="In_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=5,column='Class',value=Data['Class'].to_numpy())
data1.head()

,In_0,In_1,In_2,In_3,In_4
0,46,72,68,59,56
1,39,47,49,45,45
2,42,48,51,49,48
3,27,32,33,29,27
4,36,64,63,58,43


In [231]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

In [232]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]

    p=random.sample(range(n), math.ceil(n*0.2))
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    class_60=lst_Class.copy()
    for d in p:
        class_60[d]=5
    for i in range(n):
        A[i][i]=class_60[i]
    
    Node_class=[0,1,2,3,4]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=20
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['Class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=20
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

In [233]:

mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)
mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

Spatial only
Mean: 63.52941176470588
Standard Deviation: 4.154313354789725
Accuracy using both
Mean: 85.88235294117648
Standard Deviation: 5.212392689510297
Domain only
Mean: 91.17647058823529
Standard Deviation: 4.912841953716635


In [27]:
print(acuracy)

[84.31372549019608, 82.35294117647058, 86.27450980392157, 72.54901960784314, 88.23529411764706, 78.43137254901961, 86.27450980392157, 86.27450980392157, 86.27450980392157, 92.15686274509804]


# Cornell

In [234]:
from urllib.request import urlopen
p_data=urlopen("https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/WebKb/Cornell/out1_graph_edges.txt")
er=[]
for line in p_data:
    output = line.decode()
    er.append(output)
myls=list(er)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
print(mylist)

['173\t96', '121\t72', '35\t79', '2\t130', '2\t75', '104\t79', '104\t56', '176\t167', '176\t57', '176\t4', '176\t153', '176\t5', '176\t46', '176\t137', '176\t145', '176\t38', '176\t63', '169\t137', '169\t38', '167\t137', '167\t63', '87\t174', '48\t13', '48\t157', '28\t84', '28\t47', '57\t104', '57\t81', '57\t86', '57\t42', '57\t141', '57\t177', '57\t33', '57\t116', '57\t31', '57\t95', '57\t127', '57\t76', '57\t90', '57\t37', '57\t114', '57\t113', '57\t105', '57\t32', '57\t80', '57\t60', '57\t14', '57\t110', '57\t112', '57\t89', '57\t158', '57\t106', '57\t153', '57\t17', '57\t164', '57\t5', '57\t46', '57\t18', '57\t16', '57\t119', '57\t34', '57\t107', '57\t182', '57\t109', '57\t74', '57\t178', '57\t174', '57\t41', '57\t135', '57\t71', '57\t1', '57\t157', '57\t165', '57\t100', '57\t154', '57\t160', '57\t68', '57\t30', '57\t84', '57\t145', '57\t128', '57\t53', '57\t45', '57\t123', '57\t50', '57\t170', '57\t96', '57\t15', '57\t61', '57\t98', '57\t29', '57\t99', '57\t103', '57\t40', '57\t17

In [235]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
print(node_ID)
n=len(node_ID)
print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182]
183


In [236]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [237]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/WebKb/Cornell/out1_node_feature_label.txt',sep='\s+')
df.head()


,node_id,feature,label
0,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
1,1,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
2,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",0
3,3,"0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...",3
4,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...",3


In [238]:
fe=df["feature"].to_numpy()
lst_Class=df["label"].to_numpy()
lst_ID=df["node_id"].to_numpy()

lst_Fe=[]
for row in fe:
    d=row.split(',')
    fev=[]
    for j in range(len(d)):
            fev.append(int(d[j]))
    lst_Fe.append(fev)

In [239]:
fe_len=len(lst_Fe[0])
feature_names = ["w_{}".format(ii) for ii in range(fe_len)]
Data=pd.DataFrame(lst_Fe,columns=feature_names)
Data.insert(loc=fe_len,column='Class',value=lst_Class)
Data.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,3


In [240]:
Class=lst_Class.copy()

In [241]:
Data_0=Data.loc[Data['Class'] == 0]
Data_1=Data.loc[Data['Class'] == 1]
Data_2=Data.loc[Data['Class'] == 2]
Data_3=Data.loc[Data['Class'] == 3]
Data_4=Data.loc[Data['Class'] == 4]
Data_1.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
24,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
28,0,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,1
29,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [242]:
Basis_0=[]
Basis_1=[]
Basis_2=[]
Basis_3=[]
Basis_4=[]
for i in range(1703):
    W="w_{}".format(i)
    Basis_0.append(max(Data_0[W].to_numpy()))
    Basis_1.append(max(Data_1[W].to_numpy()))
    Basis_2.append(max(Data_2[W].to_numpy()))
    Basis_3.append(max(Data_3[W].to_numpy()))
    Basis_4.append(max(Data_4[W].to_numpy()))
#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(Basis_0)
count0=0
count1=0
for i in range(1703):
    if Basis_0[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 

In [243]:
SBasis_0=[]
SBasis_1=[]
SBasis_2=[]
SBasis_3=[]
SBasis_4=[]
#for i in range(14):
 #   W="w_{}".format(i)
  #  print(list(Data_0[W].to_numpy()).count(1))
for i in range(1703):
    W="w_{}".format(i)
    if list(Data_0[W].to_numpy()).count(1)>=int(len(Data_0.index)*0.2):
        SBasis_0.append(1)
    else:
        SBasis_0.append(0) 
    if list(Data_1[W].to_numpy()).count(1)>=int(len(Data_1.index)*0.2):
        SBasis_1.append(1)
    else:
        SBasis_1.append(0)
    if list(Data_2[W].to_numpy()).count(1)>=int(len(Data_2.index)*0.2):
        SBasis_2.append(1)
    else:
        SBasis_2.append(0)
    if list(Data_3[W].to_numpy()).count(1)>=int(len(Data_3.index)*0.2):
        SBasis_3.append(1)
    else:
        SBasis_3.append(0)
    if list(Data_4[W].to_numpy()).count(1)>=int(len(Data_4.index)*0.2):
        SBasis_4.append(1)
    else:
        SBasis_4.append(0)

#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(SBasis_3)
count0=0
count1=0
for i in range(1703):
    if SBasis_2[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 

In [244]:
Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(1703):
        if f[j]==1 and Basis_0[j]==1:
            count=count+1;
        if f[j]==1 and Basis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and Basis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and Basis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and Basis_4[j]==1:
            count4=count4+1;

    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)

    #print(f)
    f.clear()
    Fec.append(vec)
print(Fec)

[[3, 2, 3, 4, 4], [51, 44, 51, 54, 39], [35, 24, 32, 32, 24], [59, 47, 60, 70, 39], [53, 56, 63, 71, 55], [115, 93, 122, 132, 92], [129, 99, 131, 136, 156], [116, 161, 151, 147, 122], [136, 138, 175, 157, 145], [18, 17, 18, 18, 18], [42, 38, 45, 43, 48], [63, 57, 71, 86, 55], [54, 65, 64, 64, 69], [63, 59, 70, 82, 55], [43, 42, 45, 46, 36], [51, 47, 50, 62, 47], [97, 96, 129, 116, 85], [81, 95, 85, 87, 72], [34, 33, 36, 36, 33], [211, 159, 208, 206, 263], [22, 23, 21, 23, 24], [194, 175, 257, 222, 179], [165, 103, 139, 137, 97], [53, 49, 54, 58, 52], [61, 85, 79, 80, 73], [37, 28, 33, 33, 28], [36, 40, 40, 48, 29], [109, 93, 128, 114, 84], [131, 171, 155, 160, 131], [108, 147, 133, 129, 111], [105, 85, 125, 134, 91], [46, 31, 40, 40, 32], [203, 200, 276, 241, 173], [210, 133, 168, 177, 122], [54, 45, 62, 68, 48], [54, 41, 49, 51, 36], [32, 33, 31, 34, 28], [114, 104, 141, 122, 98], [41, 48, 45, 48, 41], [41, 41, 42, 50, 35], [42, 42, 45, 47, 34], [88, 70, 93, 99, 78], [108, 72, 91, 96,

In [245]:
SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0

    for j in range(1703):
        if f[j]==1 and SBasis_0[j]==1:
            count=count+1;
        if f[j]==1 and SBasis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and SBasis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and SBasis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and SBasis_4[j]==1:
            count4=count4+1;

    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)

    #print(f)
    f.clear()
    SFec.append(Svec)
print(SFec)

[[2, 2, 2, 2, 3], [30, 29, 35, 29, 30], [17, 14, 17, 11, 16], [24, 27, 30, 26, 25], [20, 40, 37, 24, 24], [39, 61, 63, 37, 51], [40, 45, 54, 28, 79], [24, 77, 84, 27, 58], [41, 74, 93, 37, 77], [16, 15, 16, 16, 15], [27, 27, 27, 20, 28], [27, 37, 39, 38, 34], [15, 30, 42, 14, 34], [31, 39, 39, 39, 38], [28, 32, 36, 32, 24], [30, 30, 32, 29, 27], [33, 65, 81, 37, 48], [31, 57, 54, 34, 36], [26, 30, 29, 30, 23], [48, 82, 86, 34, 105], [17, 20, 18, 17, 20], [50, 88, 107, 45, 87], [64, 51, 64, 34, 50], [31, 33, 40, 39, 32], [30, 57, 50, 35, 44], [26, 19, 22, 18, 19], [16, 25, 28, 27, 17], [36, 53, 73, 36, 46], [39, 88, 79, 36, 70], [35, 88, 86, 37, 59], [39, 51, 56, 47, 49], [38, 20, 21, 21, 18], [55, 105, 132, 39, 88], [86, 59, 67, 39, 53], [29, 37, 38, 31, 32], [41, 30, 31, 27, 23], [23, 28, 28, 27, 22], [32, 60, 79, 27, 52], [26, 40, 37, 28, 30], [26, 28, 30, 33, 25], [31, 32, 36, 28, 28], [47, 39, 43, 34, 47], [62, 42, 40, 31, 46], [37, 62, 69, 35, 53], [26, 20, 22, 23, 19], [30, 39, 3

In [246]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(4):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4
0,2,2,2,2,3
1,30,29,35,29,30
2,17,14,17,11,16
3,24,27,30,26,25
4,20,40,37,24,24


In [247]:
y =np.array(Fec)
data1=pd.DataFrame({"In_0":y[:,0]})
for i in range(4):
    data1.insert(loc=i+1, column="In_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=5,column='Class',value=Data['Class'].to_numpy())
data1.head()

,In_0,In_1,In_2,In_3,In_4
0,3,2,3,4,4
1,51,44,51,54,39
2,35,24,32,32,24
3,59,47,60,70,39
4,53,56,63,71,55


In [248]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]

    p=random.sample(range(n), math.ceil(n*0.2))
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    class_60=lst_Class.copy()
    for d in p:
        class_60[d]=5
    for i in range(n):
        A[i][i]=class_60[i]
    Node_class=[0,1,2,3,4,5]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Class)
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
    
    
    class_60=Class.copy()
    i=0
    for d in p:
        class_60[d]=y_pred[i]
        i=i+1
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]
    #print(A)
    Node_class=[0,1,2,3,4]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=20
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['Class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=20
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

In [249]:

mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)


Spatial Only
Mean: 52.972972972972975
Standard Deviation: 10.444235492749145
Domain only
Mean: 90.54054054054055
Standard Deviation: 5.589943083779667
Accuracy using both
Mean: 79.72972972972973
Standard Deviation: 4.809494708122212


# Texas

In [250]:
from urllib.request import urlopen
p_data=urlopen("https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/WebKb/Texas/out1_graph_edges.txt")
er=[]
for line in p_data:
    output = line.decode()
    er.append(output)
myls=list(er)
mylist = [line.rstrip('\n') for line in myls]
mylist.pop(0)
print(mylist)

['56\t84', '56\t39', '56\t9', '56\t77', '56\t89', '56\t61', '56\t125', '56\t94', '56\t64', '56\t52', '56\t27', '56\t112', '56\t21', '56\t154', '56\t31', '56\t109', '56\t57', '56\t59', '56\t22', '56\t90', '56\t68', '56\t91', '56\t63', '56\t85', '56\t116', '56\t32', '56\t143', '56\t181', '56\t54', '56\t69', '56\t3', '56\t139', '56\t104', '56\t128', '56\t137', '56\t172', '56\t93', '56\t179', '56\t132', '56\t134', '56\t178', '56\t144', '56\t74', '56\t46', '56\t11', '56\t80', '56\t124', '56\t135', '56\t97', '56\t98', '56\t79', '56\t23', '56\t118', '56\t167', '56\t131', '56\t153', '56\t33', '56\t149', '56\t161', '56\t127', '56\t34', '56\t136', '56\t169', '56\t150', '56\t62', '56\t26', '56\t82', '56\t12', '56\t47', '56\t166', '56\t171', '56\t6', '56\t17', '56\t53', '56\t117', '56\t87', '56\t146', '56\t49', '56\t159', '56\t119', '56\t170', '56\t38', '56\t55', '56\t30', '56\t165', '56\t103', '56\t155', '56\t110', '56\t16', '56\t44', '56\t75', '56\t41', '56\t29', '56\t35', '56\t65', '56\t19', '5

In [126]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/joshem163/Node-Classification-Graph/main/DataSet/WebKb/Cornell/out1_graph_edges.txt', sep='\s+', header=None)
#df.to_csv('my_file.csv', header=None)
df.head()

,0,1
0,node_id,node_id
1,173,96
2,121,72
3,35,79
4,2,130


In [251]:
mynode=[]
i=0
for d in mylist:
    d=mylist[i].split("\t")
    mynode.append(int(d[0]))
    mynode.append(int(d[1]))
    i=i+1
#print(mynode)
node_ID1=np.unique(mynode)
node_ID=list(node_ID1)
print(node_ID)
n=len(node_ID)
print(n)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182]
183


In [252]:
A=np.zeros((n,n))
i=0
for d in mylist:
    d=mylist[i].split("\t")
    A[node_ID.index(int(d[0]))][node_ID.index(int(d[1]))]=1
    i=i+1;
print(" The adjacency Matrix")
print("=======================")
print(A)

 The adjacency Matrix
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [253]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/class-contrast/NodeClassification/main/Datasets/WebKb/Texas/out1_node_feature_label.txt',sep='\s+')
#df.to_csv('my_file.csv', header=None)
df.head()

,node_id,feature,label
0,0,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
1,1,"0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,...",0
2,2,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",2
3,3,"0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",3
4,4,"0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...",4


In [254]:
fe=df["feature"].to_numpy()
lst_Class=df["label"].to_numpy()
lst_ID=df["node_id"].to_numpy()

lst_Fe=[]
for row in fe:
    d=row.split(',')
    fev=[]
    for j in range(len(d)):
            fev.append(int(d[j]))
    lst_Fe.append(fev)

In [256]:
fe_len=len(lst_Fe[0])
feature_names = ["w_{}".format(ii) for ii in range(fe_len)]
Data=pd.DataFrame(lst_Fe,columns=feature_names)
Data.insert(loc=fe_len,column='Class',value=lst_Class)
Data.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4


In [257]:
Class=lst_Class.copy()
Data_0=Data.loc[Data['Class'] == 0]
Data_1=Data.loc[Data['Class'] == 1]
Data_2=Data.loc[Data['Class'] == 2]
Data_3=Data.loc[Data['Class'] == 3]
Data_4=Data.loc[Data['Class'] == 4]
Data_1.head()

,w_0,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,...,w_1694,w_1695,w_1696,w_1697,w_1698,w_1699,w_1700,w_1701,w_1702,Class
25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [260]:
Basis_0=[]
Basis_1=[]
Basis_2=[]
Basis_3=[]
Basis_4=[]
for i in range(fe_len):
    W="w_{}".format(i)
    Basis_0.append(max(Data_0[W].to_numpy()))
    Basis_1.append(max(Data_1[W].to_numpy()))
    Basis_2.append(max(Data_2[W].to_numpy()))
    Basis_3.append(max(Data_3[W].to_numpy()))
    Basis_4.append(max(Data_4[W].to_numpy()))
#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(Basis_0)
count0=0
count1=0
for i in range(fe_len):
    if Basis_0[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 

In [261]:
SBasis_0=[]
SBasis_1=[]
SBasis_2=[]
SBasis_3=[]
SBasis_4=[]
for i in range(fe_len):
    W="w_{}".format(i)
    if list(Data_0[W].to_numpy()).count(1)>=int(len(Data_0.index)*0.2):
        SBasis_0.append(1)
    else:
        SBasis_0.append(0) 
    if list(Data_1[W].to_numpy()).count(1)>=int(len(Data_1.index)*0.2):
        SBasis_1.append(1)
    else:
        SBasis_1.append(0)
    if list(Data_2[W].to_numpy()).count(1)>=int(len(Data_2.index)*0.2):
        SBasis_2.append(1)
    else:
        SBasis_2.append(0)
    if list(Data_3[W].to_numpy()).count(1)>=int(len(Data_3.index)*0.2):
        SBasis_3.append(1)
    else:
        SBasis_3.append(0)
    if list(Data_4[W].to_numpy()).count(1)>=int(len(Data_4.index)*0.2):
        SBasis_4.append(1)
    else:
        SBasis_4.append(0)

#W1=Data_0['w_0'].to_numpy()
#print(max(W1))
print(SBasis_3)
count0=0
count1=0
for i in range(fe_len):
    if SBasis_2[i]==1:
        count1=count1+1
    else:
        count0=count0+1
print(count0)
print(count1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [263]:
Fec=[]
for i in range(n):
    vec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0
    for j in range(fe_len):
        if f[j]==1 and Basis_0[j]==1:
            count=count+1;
        if f[j]==1 and Basis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and Basis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and Basis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and Basis_4[j]==1:
            count4=count4+1;

    vec.append(count)
    vec.append(count1)
    vec.append(count2)
    vec.append(count3)
    vec.append(count4)

    #print(f)
    f.clear()
    Fec.append(vec)
print(Fec)

[[39, 20, 40, 46, 40], [231, 25, 155, 192, 161], [25, 20, 26, 26, 26], [61, 17, 50, 77, 64], [81, 29, 62, 77, 88], [85, 33, 68, 100, 83], [81, 20, 56, 72, 61], [56, 14, 39, 49, 40], [75, 21, 63, 93, 75], [46, 22, 39, 44, 36], [32, 22, 31, 39, 37], [72, 34, 65, 77, 69], [64, 28, 56, 77, 62], [72, 33, 65, 80, 74], [128, 31, 104, 154, 118], [133, 34, 111, 142, 160], [127, 26, 100, 157, 116], [58, 30, 58, 66, 63], [54, 17, 32, 46, 40], [59, 27, 47, 79, 51], [280, 26, 174, 223, 206], [53, 28, 53, 65, 56], [43, 25, 41, 46, 44], [57, 30, 48, 66, 57], [77, 25, 63, 104, 72], [43, 45, 41, 45, 44], [64, 15, 55, 88, 66], [90, 25, 80, 122, 86], [19, 14, 19, 19, 19], [125, 25, 148, 125, 115], [49, 25, 47, 59, 53], [61, 31, 54, 69, 62], [84, 28, 70, 86, 99], [45, 28, 42, 50, 46], [70, 26, 67, 82, 87], [48, 24, 43, 52, 56], [61, 21, 68, 62, 60], [65, 24, 57, 79, 63], [41, 24, 40, 45, 41], [50, 28, 45, 58, 51], [143, 19, 86, 121, 89], [100, 26, 80, 116, 126], [90, 17, 62, 78, 66], [64, 25, 55, 77, 60],

In [264]:
SFec=[]
for i in range(n):
    Svec=[]
    f=Data.loc[i, feature_names].values.flatten().tolist()
    count=0
    count1=0
    count2=0
    count3=0
    count4=0

    for j in range(1703):
        if f[j]==1 and SBasis_0[j]==1:
            count=count+1;
        if f[j]==1 and SBasis_1[j]==1:
            count1=count1+1;
        if f[j]==1 and SBasis_2[j]==1:
            count2=count2+1;
        if f[j]==1 and SBasis_3[j]==1:
            count3=count3+1;
        if f[j]==1 and SBasis_4[j]==1:
            count4=count4+1;

    Svec.append(count)
    Svec.append(count1)
    Svec.append(count2)
    Svec.append(count3)
    Svec.append(count4)

    #print(f)
    f.clear()
    SFec.append(Svec)
print(SFec)

[[24, 46, 27, 24, 27], [101, 231, 75, 37, 60], [20, 26, 24, 21, 22], [30, 77, 29, 31, 38], [45, 88, 48, 37, 56], [46, 100, 47, 43, 52], [54, 81, 37, 29, 37], [38, 56, 25, 16, 21], [32, 93, 35, 34, 35], [36, 46, 29, 28, 27], [24, 39, 24, 32, 31], [45, 77, 51, 46, 47], [36, 77, 34, 31, 37], [44, 80, 50, 44, 46], [53, 154, 56, 44, 52], [51, 160, 60, 42, 69], [53, 157, 55, 37, 51], [39, 66, 45, 44, 46], [33, 54, 21, 21, 24], [28, 79, 32, 33, 30], [101, 280, 76, 39, 68], [34, 65, 40, 38, 40], [28, 46, 35, 31, 33], [37, 66, 39, 40, 40], [38, 104, 41, 34, 38], [27, 45, 39, 33, 34], [23, 88, 31, 24, 31], [43, 122, 46, 35, 43], [19, 19, 18, 15, 18], [47, 148, 68, 32, 47], [32, 59, 38, 32, 35], [38, 69, 40, 43, 44], [38, 99, 47, 30, 46], [33, 50, 31, 37, 38], [40, 87, 40, 34, 47], [26, 56, 29, 23, 32], [34, 68, 48, 26, 36], [35, 79, 37, 30, 39], [28, 45, 32, 31, 32], [36, 58, 37, 35, 41], [72, 143, 41, 28, 37], [46, 126, 48, 40, 58], [44, 90, 37, 23, 29], [34, 77, 35, 32, 34], [31, 45, 32, 30, 3

In [265]:
z =np.array(SFec)
data2=pd.DataFrame({"S_0":z[:,0]})
for i in range(4):
    data2.insert(loc=i+1, column="S_{}".format(i+1), value=z[:,i+1])
#data2.insert(loc=7,column='Class',value=Data['subject'].to_numpy())
data2.head()

,S_0,S_1,S_2,S_3,S_4
0,24,46,27,24,27
1,101,231,75,37,60
2,20,26,24,21,22
3,30,77,29,31,38
4,45,88,48,37,56


In [266]:
y =np.array(Fec)
data1=pd.DataFrame({"In_0":y[:,0]})
for i in range(4):
    data1.insert(loc=i+1, column="In_{}".format(i+1), value=y[:,i+1])
#data1.insert(loc=5,column='Class',value=Data['Class'].to_numpy())
data1.head()

,In_0,In_1,In_2,In_3,In_4
0,39,20,40,46,40
1,231,25,155,192,161
2,25,20,26,26,26
3,61,17,50,77,64
4,81,29,62,77,88


In [281]:
Ac_fe=[]
acuracy=[]
Ac_domain2=[]
Ac_domain=[]

for it in range(10):
    p=[]
    q=[]

    p=random.sample(range(n), math.ceil(n*0.2))
    q=[]
    for i in range(n):
        if i not in p:
            q.append(i)
    if 25 in p:
        q.append(25)
        p.remove(25)
    class_60=lst_Class.copy()
    for d in p:
        class_60[d]=5
    for i in range(n):
        A[i][i]=class_60[i]
    Node_class=[0,1,2,3,4,5]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Class)
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

 
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
# Model Accuracy, how often is the classifier correct?
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred)*100,"%")
    
    
    class_60=Class.copy()
    i=0
    for d in p:
        class_60[d]=y_pred[i]
        i=i+1
    #print(class_60)
    for i in range(n):
        A[i][i]=class_60[i]
    #print(A)
    Node_class=[0,1,2,3,4]
    F_vec=spatial_two (Node_class, A,n)
    x =np.array(F_vec)
    k=len(F_vec[0])
    data=pd.DataFrame({"a_0":x[:,0]})
    data.insert(loc=1, column="b_0", value=x[:,1])
    for i in range(1,k-1,2):
        data.insert(loc=i+1, column="a_{}".format(int((i+1)/2)), value=x[:,i+1])
        data.insert(loc=i+2, column="b_{}".format(int((i+1)/2)), value=x[:,i+2])

    data.insert(loc=k,column='Class',value=Data['Class'].to_numpy())
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))

    X=data[feature] # Features
    y=data['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    Ac_domain2.append(metrics.accuracy_score(y_test, y_pred)*100)
    result = pd.concat([data2,data1,data], axis=1)
    k=20
    feature=["a_0","b_0"]
    for i in range(1,k-1,2):
        feature.append("a_{}".format(int((i+1)/2)))
        feature.append("b_{}".format(int((i+1)/2)))
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]

    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    
    acuracy.append(metrics.accuracy_score(y_test, y_pred)*100)
    feature=[]
    for i in range(4):
        feature.append("S_{}".format(i))
    for j in range(4):
        feature.append("In_{}".format(j))
    #print(feature)
    X=result[feature] # Features
    y=result['Class']  # Labels
    X_train=X.iloc[q]
    X_test=X.iloc[p]
    y_train=y.iloc[q]
    y_test=y.iloc[p]
    scaler = StandardScaler()  
# Don't cheat - fit only on training data
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
# apply same transformation to test data
    X_test = scaler.transform(X_test)  
    clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(700,), random_state=1,max_iter=10000, warm_start=True)
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    Ac_fe.append(metrics.accuracy_score(y_test, y_pred)*100)

In [282]:

mean_value = statistics.mean(Ac_domain2)
std_dev = statistics.stdev(Ac_domain2)
print("Spatial Only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(Ac_fe)
std_dev = statistics.stdev(Ac_fe)
print("Domain only")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)

mean_value = statistics.mean(acuracy)
std_dev = statistics.stdev(acuracy)
print("Accuracy using both")
print("Mean:", mean_value)
print("Standard Deviation:", std_dev)


Spatial Only
Mean: 74.27177177177177
Standard Deviation: 4.898267505566394
Domain only
Mean: 89.96246246246247
Standard Deviation: 5.871667649440834
Accuracy using both
Mean: 85.11261261261261
Standard Deviation: 5.2600257198302725
